In [1]:
%load_ext autoreload
%autoreload 2
import os

import torch
torch.manual_seed(42)
import networkx as nx
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from nn_homology import nn_graph

In [2]:
# instantiate simple lenet model
class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64*14*14, 256),
#             nn.Linear(50176, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )

        self.param_info = [{'layer_type': 'Conv2d', 'kernel_size':(3,3), 'stride':1, 'padding': 1, 'name':'Conv1'},
                            {'layer_type': 'Conv2d', 'kernel_size':(3,3), 'stride':1, 'padding':1, 'name':'Conv2'},
                            {'layer_type':'MaxPool2d', 'kernel_size':(2,2), 'stride':2, 'padding':0, 'name':'MaxPool1'},
                            {'layer_type':'Linear', 'name': 'Linear1'},
                            {'layer_type':'Linear', 'name': 'Linear2'}]


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
# get train and test data loaders
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=60, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=60, shuffle=True)

In [5]:
# set up model for training, instantiate optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)

In [6]:
# train 1 epoch to get some non-random model performance and weights
for epoch in range(1,2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304141
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.604650
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.766256
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.754349
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.444256
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.362328
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.380842
Train Epoch: 1 [42000/60000 (70%)]	Loss: 0.223329
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.357630
Train Epoch: 1 [54000/60000 (90%)]	Loss: 0.208049

Test set: Average loss: 0.2763, Accuracy: 9198/10000 (92%)



## Parameter Graph

In [7]:
# expected size of input layer
input_size = (1,1,28,28)
# load the model class (to access param_info)
mc = LeNet5()

In [8]:
# compute parameter graph (directed)
NNG = nn_graph.NNGraph(undirected=False)
%time NNG.parameter_graph(model, mc.param_info, input_size, ignore_zeros=True, verbose=True)

Layer: Conv1
Layer: Conv2
Layer: MaxPool1
Layer: Linear1
Layer: Linear2
CPU times: user 3min 5s, sys: 2.23 s, total: 3min 8s
Wall time: 3min 4s


In [9]:
# sanity check longest path in parameter network
nx.dag_longest_path(NNG.G)

['Conv1_0_783',
 'Conv2_11_782',
 'MaxPool1_29_783',
 'Linear1_0_5879',
 'Linear2_0_4',
 'Output_0_1']

## Activation Graph

In [10]:
# instantiate new test loader with batch size 1
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
testdataset = datasets.MNIST('../../data', train=False, transform=transform)
test_loader = torch.utils.data.DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=0)

In [11]:
# make sure data and model are on same device
device = torch.device("cpu")
model = model.to(device)

# our batch size is 1, so compute activation network for single input
data,target = next(iter(test_loader))
data = data.to(device)

In [12]:
# compute activation graph (will be directed).
# use absolute value weight function to see if we capture the 
# correct class in the activation network. This assumes that
# highly negative activations aren't affecting the classification 
# of this particular input.
%time G_act = nn_graph.activation_graph(model, model.param_info, data, ignore_zeros=False, verbose=True, weight_func=lambda x: np.abs(x))

Layer: Conv1
Layer: Conv2
Layer: MaxPool1
Layer: Linear1
Layer: Linear2
CPU times: user 2min 25s, sys: 3.06 s, total: 2min 28s
Wall time: 2min 25s


In [13]:
# sanity check longest path
nx.dag_longest_path(G_act)

['Conv1_0_739',
 'Conv2_63_739',
 'MaxPool1_24_767',
 'Linear1_0_4891',
 'Linear2_0_220',
 'Output_0_7']

In [14]:
target

tensor([7])